<a href="https://colab.research.google.com/github/nikolmash/yes-no-question-answering/blob/master/QA_bert_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install jsonlines
import numpy as np
import pandas as pd
import tensorflow as tf 
import jsonlines
from tensorflow import keras
from tensorflow.keras import layers
from transformers import *
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score

     |████████████████████████████████| 665kB 6.7MB/s 
     |████████████████████████████████| 3.8MB 21.8MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 
     |████████████████████████████████| 1.1MB 37.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a92f3f4bf9732b94ed15598a3042d0c12b156ba959d253f2758498ba8bf9b287
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with jsonlines.open('/content/drive/My Drive/train.jsonl') as reader:
      train = [obj for obj in reader]
with jsonlines.open('/content/drive/My Drive/dev.jsonl') as reader:
      dev = [obj for obj in reader]

In [4]:
config = BertConfig(num_labels=2)

bert_model = TFBertModel.from_pretrained("bert-base-uncased", config = config) 
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [0]:
train_df = pd.DataFrame(train)
dev_df = pd.DataFrame(dev)

In [0]:
train_df['question'] = train_df['question'].apply(bert_tokenizer.tokenize)
train_df['passage'] = train_df['passage'].apply(bert_tokenizer.tokenize)
dev_df['question'] = dev_df['question'].apply(bert_tokenizer.tokenize)
dev_df['passage'] = dev_df['passage'].apply(bert_tokenizer.tokenize)

In [0]:
max_length = 512
train_input_ids = []
train_attention_masks = []
train_token_ids = []

for question, text in zip(train_df['question'].values, train_df['passage'].values):
  encoded_data = bert_tokenizer.encode_plus(question, text, max_length=max_length, add_special_tokens=True,
                                            pad_to_max_length=True, truncation_strategy="longest_first")
  input_ids = encoded_data['input_ids']
  attention_mask = encoded_data['attention_mask']
  token_type_ids = encoded_data['token_type_ids']
  train_input_ids.append(input_ids)
  train_attention_masks.append(attention_mask)
  train_token_ids.append(token_type_ids)

train_input_ids = np.array(train_input_ids)
train_attention_masks = np.array(train_attention_masks)
train_token_ids = np.array(train_token_ids)

In [0]:
dev_input_ids = []
dev_attention_masks = []
dev_token_ids = []

for question, text in zip(dev_df['question'].values, dev_df['passage'].values):
  encoded_data = bert_tokenizer.encode_plus(question, text, max_length=max_length, add_special_tokens=True,
                                            pad_to_max_length=True, truncation_strategy="longest_first")
  input_ids = encoded_data['input_ids']
  attention_mask = encoded_data['attention_mask']
  token_type_ids = encoded_data['token_type_ids']
  dev_input_ids.append(input_ids)
  dev_attention_masks.append(attention_mask)
  dev_token_ids.append(token_type_ids)

dev_input_ids = np.array(dev_input_ids)
dev_attention_masks = np.array(dev_attention_masks)
dev_token_ids = np.array(dev_token_ids)

In [0]:
train_answers = np.array(train_df['answer'].values.astype(int))
dev_answers = dev_df['answer'].values.astype(int)

In [0]:
input_ids = layers.Input(shape=(max_length,), dtype=tf.int32)
token_type_ids = layers.Input(shape=(max_length,), dtype=tf.int32)
attention_mask = layers.Input(shape=(max_length,), dtype=tf.int32)

embedding_layer = bert_model(input_ids, token_type_ids=token_type_ids,
                             attention_mask=attention_mask)[0]

X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False))(embedding_layer)
X = tf.keras.layers.Dense(32, activation='relu')(X)
X = tf.keras.layers.Dropout(0.3)(X)
X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs = X)

In [45]:
for layer in model.layers[:4]:
  layer.trainable = False

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_33 (InputLayer)           [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109482240   input_31[0][0]                   
___________________________________________________________________________________________

In [46]:
model.fit(x=[train_input_ids, train_token_ids, train_attention_masks],
          y=train_answers,
          batch_size=32,
          epochs=15,
          steps_per_epoch=100,
          validation_split=0.1,
          verbose=1)

Epoch 1/15
100/100 [==============================] - 108s 1s/step - loss: 0.6685 - accuracy: 0.6184 - val_loss: 0.6749 - val_accuracy: 0.5938
Epoch 2/15
100/100 [==============================] - 104s 1s/step - loss: 0.6549 - accuracy: 0.6244 - val_loss: 0.6714 - val_accuracy: 0.5928
Epoch 3/15
100/100 [==============================] - 103s 1s/step - loss: 0.6432 - accuracy: 0.6381 - val_loss: 0.6710 - val_accuracy: 0.5949
Epoch 4/15
100/100 [==============================] - 104s 1s/step - loss: 0.6276 - accuracy: 0.6444 - val_loss: 0.6592 - val_accuracy: 0.6055
Epoch 5/15
100/100 [==============================] - 104s 1s/step - loss: 0.6284 - accuracy: 0.6488 - val_loss: 0.6503 - val_accuracy: 0.6151
Epoch 6/15
100/100 [==============================] - 103s 1s/step - loss: 0.6280 - accuracy: 0.6532 - val_loss: 0.6596 - val_accuracy: 0.6278
Epoch 7/15
100/100 [==============================] - 104s 1s/step - loss: 0.6004 - accuracy: 0.6834 - val_loss: 0.6508 - val_accuracy: 0.6002

In [0]:
preds = model.predict([dev_input_ids, dev_token_ids, dev_attention_masks])

In [0]:
labels = (preds > 0.5).astype(np.int)

In [56]:
print('Accuracy: ', accuracy_score(dev_answers, labels))
print('F1 score: ', f1_score(dev_answers, labels))


Accuracy:  0.6675840978593273
F1 score:  0.7562233684682663
